# Pre-processing of Data Sets

In [1]:
import pandas as pd
from collections import namedtuple
import numpy as np
import time
import pickle
from importlib import reload

import sys
sys.path.insert(0, '../')
import bench_util

%load_ext line_profiler

## Utility Data

In [3]:
# Read the CSV file and convert the billing period dates into 
# real Pandas dates
dfu = pd.read_csv('data/AllDataExport.csv', parse_dates=['From', 'Thru'])

# Pickle it for use in the other notebook
dfu.to_pickle('df_raw.pkl')
dfu.head()

,Site ID,Site Name,Vendor Code,Vendor Name,Account Number,Bill Date,Due Date,Entry Date,Invoice #,Voucher #,From,Thru,Service Name,Item Description,Meter Number,Usage,Cost,Units,Account Financial Code,Site Financial Code
0,TRGR,FNSB-Transit Garage,VP287678,Sourdough Fuel (Petro Star),00013297 (closed),09/28/2010,09/28/2010,01/26/2011,NaN,NaN,2008-11-19,2010-09-28,Oil #1,FED LUS TX,NaN,NaN,3.00,NaN,NaN,NaN
1,TRGR,FNSB-Transit Garage,VP287678,Sourdough Fuel (Petro Star),00013297 (closed),09/28/2010,09/28/2010,01/26/2011,NaN,NaN,2008-11-19,2010-09-28,Oil #1,Fuel Oil #1 (Gallons),NaN,3000.0,7950.00,Gallons,NaN,NaN
2,TRGR,FNSB-Transit Garage,VP287678,Sourdough Fuel (Petro Star),00013297 (closed),09/30/2010,09/30/2010,01/26/2011,NaN,NaN,2010-09-28,2010-09-30,Oil #1,FED LUS TX,NaN,NaN,1.31,NaN,NaN,NaN
3,TRGR,FNSB-Transit Garage,VP287678,Sourdough Fuel (Petro Star),00013297 (closed),09/30/2010,09/30/2010,01/26/2011,NaN,NaN,2010-09-28,2010-09-30,Oil #1,Fuel Oil #1 (Gallons),NaN,1307.0,3463.82,Gallons,NaN,NaN
4,TRGR,FNSB-Transit Garage,VP287678,Sourdough Fuel (Petro Star),00013297 (closed),01/14/2011,01/14/2011,07/28/2014,NaN,NaN,2010-09-30,2011-01-14,Oil #1,Fuel Oil #1 (Gallons),NaN,1880.0,5545.41,Gallons,NaN,NaN


In [4]:
len(dfu)

71702

In [22]:
# Make a utility function object
reload(bench_util)
ut = bench_util.Util(dfu, '../data/Other_Building_Data.xlsx')

In [23]:
cols = ['Site ID', 'Vendor Code', 'Vendor Name', 'Account Number', 'Service Name', 'Item Description',
       'Meter Number', 'Units', 'Account Financial Code', 'Site Financial Code']
for col in cols:
    print('{0:24s}: {1}'.format(col, list(dfu[col].unique())))

Site ID                 : ['TRGR', 'CLXGP2', 'CLXES1', 'CLXSO1', 'CLXSM1', '11', 'TRPBG1', 'NWLBG1', '05', 'HSPSWP', '15A', 'BAOBG1', '15', '15B', 'DIPMP1', 'ANSBG1', 'MSRSWP', 'PRW', '03', '06', '09', '42', '04', '104', '13', '27', '28', '29', '44', '40', '47', '07', '08', 'CLX001', 'CLX002', 'CLX003', 'CLX004', 'VMP001', 'TRPAIR', 'GFP001', 'CACBG1', 'HEZ001', 'KWP001', 'ASLELC1', '23', 'BALHHW', '12', 'KIP001', 'HEMBG1', '45', '22', 'WSPSWP', 'GSWNP', 'BHPCCS', 'NPP001', 'GRP001', 'TRANS10', '14', '10', 'SHW001', 'TRANS06', 'ASLPL1', '49', 'TRANS09', 'BAP001', 'ASLELC2', 'KEP001', 'NWLP01', 'ASLGP2', 'CBS001', 'MTP001', 'WSPP01', 'GF001', 'ASLTVR', 'ASLCHU', 'ASLCV1', 'ASLC18', 'ASLPIH', 'ASLHIS', 'ASLSEA', 'ASLC21', 'CSP001', 'WF001', 'MF001', 'MSLL001', '76', '37', 'GRPLFT', 'BHPSKI2', 'DOGPRK', '34', 'MNPPRK', 'SF001', 'NBP001', 'NWP001', 'NRP001', 'MSWBG2', 'MSWBG1', 'LF001', 'BENBG1', 'LEABG1', 'STRBG1', 'CRB001', 'BHPSKI3', 'BHPSKI4', 'ASLGDM', 'ASLSQD', 'ASLC47', 'WSPGAR', 'A

In [24]:
dfu[dfu['Service Name']=="Oil #2"]

,Site ID,Site Name,Vendor Code,Vendor Name,Account Number,Bill Date,Due Date,Entry Date,Invoice #,Voucher #,From,Thru,Service Name,Item Description,Meter Number,Usage,Cost,Units,Account Financial Code,Site Financial Code
18,TRGR,FNSB-Transit Garage,VP287678,Sourdough Fuel (Petro Star),00013297 (closed),04/11/2011,04/11/2011,11/19/2013,NaN,NaN,2011-03-12,2011-04-11,Oil #2,Tax: Regulatory,NaN,NaN,3.6,NaN,NaN,NaN


In [7]:
# Save out the Unique Site IDs and Names
#df_sites = pd.DataFrame(data=list(set(zip(dfu['Site ID'], dfu['Site Name']))))
#df_sites.to_excel('sites.xlsx')

In [30]:
# Filter down to the needed columns and rename them
cols = [
    ('Site ID', 'site_id'),
    ('From', 'from_dt'),
    ('Thru', 'thru_dt'),
    ('Service Name', 'service_type'),
    ('Item Description', 'item_desc'),
    ('Usage', 'usage'),
    ('Cost', 'cost'),
    ('Units', 'units'),
]

old_cols, new_cols = zip(*cols)         # unpack into old and new column names
dfu1 = dfu[list(old_cols)].copy()              # select just those columns from the origina dataframe
dfu1.columns = new_cols                 # rename the columns
dfu1.head()

,site_id,from_dt,thru_dt,service_type,item_desc,usage,cost,units
0,TRGR,2008-11-19,2010-09-28,Oil #1,FED LUS TX,NaN,3.00,NaN
1,TRGR,2008-11-19,2010-09-28,Oil #1,Fuel Oil #1 (Gallons),3000.0,7950.00,Gallons
2,TRGR,2010-09-28,2010-09-30,Oil #1,FED LUS TX,NaN,1.31,NaN
3,TRGR,2010-09-28,2010-09-30,Oil #1,Fuel Oil #1 (Gallons),1307.0,3463.82,Gallons
4,TRGR,2010-09-30,2011-01-14,Oil #1,Fuel Oil #1 (Gallons),1880.0,5545.41,Gallons


In [31]:
dfu1.query('service_type == "Oil #2"')

,site_id,from_dt,thru_dt,service_type,item_desc,usage,cost,units
18,TRGR,2011-03-12,2011-04-11,Oil #2,Tax: Regulatory,NaN,3.6,NaN


In [32]:
# Unique sets of service_type and units
df_usage = dfu1.query('usage > 0')
set(zip(df_usage.service_type, df_usage.units))

{('Electricity', 'kVAR'),
 ('Electricity', 'kVARh'),
 ('Electricity', 'kW'),
 ('Electricity', 'kWh'),
 ('Natural Gas', 'CCF'),
 ('Oil #1', 'Gallons'),
 ('Refuse', 'Loads'),
 ('Refuse', 'Tons'),
 ('Sewer', 'Gallons'),
 ('Steam', 'MMBtu'),
 ('Steam', 'klbs'),
 ('Steam', 'lbs'),
 ('Water', 'Cgallons'),
 ('Water', 'Gallons'),
 ('Water', 'kGal')}

In [33]:
df_usage.query('service_type == "Electricity" and units == "kVARh"')

,site_id,from_dt,thru_dt,service_type,item_desc,usage,cost,units
45452,PRW,2012-09-21,2012-10-19,Electricity,kVARh/Excess kVARh,1.0,0.0,kVARh


### Collapse Non-Usage Changes into "Other Charge"

In [34]:
# Back to processing the main utility bill DataFrame

# Now collapse all the non-usage charges into one item_desc: Other Charge
# This cuts the processing time in half due to not having to split a whole 
# bunch of non-consumption charges.
dfu1.loc[np.isnan(dfu1.usage), 'item_desc'] = 'Other Charge'
dfu1.units.fillna('-', inplace=True)   # Pandas can't do a GroupBy on NaNs, so replace with something
dfu1 = dfu1.groupby(['site_id', 'from_dt', 'thru_dt', 'service_type', 'item_desc', 'units']).sum()
dfu1.reset_index(inplace=True)
dfu1.head(20)

,site_id,from_dt,thru_dt,service_type,item_desc,units,usage,cost
0,03,2010-07-13,2010-08-11,Electricity,Electricity charge,-,0.0,20.00
1,03,2010-07-15,2010-08-13,Electricity,Electricity charge,-,0.0,20.00
2,03,2010-07-22,2010-08-20,Electricity,Electricity charge,kWh,20320.0,3981.47
3,03,2010-07-29,2010-08-30,Natural Gas,Natural gas (CCF),-,0.0,0.00
4,03,2010-07-29,2010-08-30,Natural Gas,Other Charge,-,NaN,17.31
5,03,2010-07-29,2010-09-02,Sewer,Other Charge,-,NaN,257.36
6,03,2010-07-29,2010-09-02,Water,Other Charge,-,NaN,75.32
7,03,2010-07-29,2010-09-02,Water,Water Usage (Gallons),Gallons,23800.0,235.59
8,03,2010-07-31,2010-08-31,Refuse,Other Charge,-,NaN,57.59
9,03,2010-07-31,2010-08-31,Refuse,Refuse (Loads),Loads,7.0,297.43


### Split Each Bill into Multiple Pieces, each within one Calendar Month

In [35]:
# Test the split_period function
bench_util.split_period('2016-01-25', '2016-06-26')
# this takes about 3.5 msec to due, which is pretty long

[PeriodSplit(cal_year=2016, cal_mo=1, bill_frac=0.042483660130718956, days_served=6.5),
 PeriodSplit(cal_year=2016, cal_mo=2, bill_frac=0.18954248366013071, days_served=29.0),
 PeriodSplit(cal_year=2016, cal_mo=3, bill_frac=0.20261437908496732, days_served=31.0),
 PeriodSplit(cal_year=2016, cal_mo=4, bill_frac=0.19607843137254902, days_served=30.0),
 PeriodSplit(cal_year=2016, cal_mo=5, bill_frac=0.20261437908496732, days_served=31.0),
 PeriodSplit(cal_year=2016, cal_mo=6, bill_frac=0.16666666666666666, days_served=25.5)]

In [36]:
# Split all the rows into calendar month pieces and make a new DataFrame
recs=[]
for ix, row in dfu1.iterrows():
    # it is *much* faster to modify a dictionary than a Pandas series
    row_tmpl = row.to_dict()   

    # Pull out start and end of billing period; can drop the from & thru dates now
    # doing split-up of billing period across months.
    st = row_tmpl['from_dt']
    en = row_tmpl['thru_dt']
    del row_tmpl['from_dt']
    del row_tmpl['thru_dt']
    
    for piece in bench_util.split_period(st, en):
        new_row = row_tmpl.copy()
        new_row['cal_year'] = piece.cal_year
        new_row['cal_mo'] = piece.cal_mo
        new_row['days_served'] = piece.days_served
        new_row['usage'] *= piece.bill_frac
        new_row['cost'] *= piece.bill_frac
        recs.append(new_row)

dfu2 = pd.DataFrame(recs, index=range(len(recs)))
dfu2.head()

,cal_mo,cal_year,cost,days_served,item_desc,service_type,site_id,units,usage
0,7,2010,12.758621,18.5,Electricity charge,Electricity,03,-,0.000000
1,8,2010,7.241379,10.5,Electricity charge,Electricity,03,-,0.000000
2,7,2010,11.379310,16.5,Electricity charge,Electricity,03,-,0.000000
3,8,2010,8.620690,12.5,Electricity charge,Electricity,03,-,0.000000
4,7,2010,1304.274655,9.5,Electricity charge,Electricity,03,kWh,6656.551724


### Sum Up the Pieces by Month

In [37]:
dfu3 = dfu2.groupby(
    ['site_id', 'service_type', 'cal_year', 'cal_mo', 'item_desc', 'units']
).sum()
dfu3 = dfu3.reset_index()
dfu3.head(10)

,site_id,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage
0,03,Electricity,2010,7,Electricity charge,-,24.137931,35.0,0.000000
1,03,Electricity,2010,7,Electricity charge,kWh,1304.274655,9.5,6656.551724
2,03,Electricity,2010,8,Electricity charge,-,39.498433,62.0,0.000000
3,03,Electricity,2010,8,Electricity charge,kWh,4940.646254,31.0,26822.236155
4,03,Electricity,2010,9,Electricity charge,-,16.363636,27.0,0.000000
5,03,Electricity,2010,9,Electricity charge,kWh,6206.509032,63.0,36144.268383
6,03,Electricity,2010,10,Electricity charge,kWh,7637.579025,102.0,45079.667877
7,03,Electricity,2010,11,Electricity charge,kWh,8051.234399,90.0,47705.533154
8,03,Electricity,2010,12,Electricity charge,kWh,7733.783390,93.0,44821.714849
9,03,Electricity,2011,1,Electricity charge,kWh,6295.576913,93.0,35977.482508


In [38]:
dfu3[dfu3.service_type=='Electricity'].head(10)

,site_id,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage
0,03,Electricity,2010,7,Electricity charge,-,24.137931,35.0,0.000000
1,03,Electricity,2010,7,Electricity charge,kWh,1304.274655,9.5,6656.551724
2,03,Electricity,2010,8,Electricity charge,-,39.498433,62.0,0.000000
3,03,Electricity,2010,8,Electricity charge,kWh,4940.646254,31.0,26822.236155
4,03,Electricity,2010,9,Electricity charge,-,16.363636,27.0,0.000000
5,03,Electricity,2010,9,Electricity charge,kWh,6206.509032,63.0,36144.268383
6,03,Electricity,2010,10,Electricity charge,kWh,7637.579025,102.0,45079.667877
7,03,Electricity,2010,11,Electricity charge,kWh,8051.234399,90.0,47705.533154
8,03,Electricity,2010,12,Electricity charge,kWh,7733.783390,93.0,44821.714849
9,03,Electricity,2011,1,Electricity charge,kWh,6295.576913,93.0,35977.482508


### Add Fiscal Year Info and MMBtus

In [39]:
# Add Fiscal Year and month columns
fyr = []
fmo = []
for cyr, cmo in zip(dfu3.cal_year, dfu3.cal_mo):
    fis_yr, fis_mo = bench_util.calendar_to_fiscal(cyr, cmo)
    fyr.append(fis_yr)
    fmo.append(fis_mo)
dfu3['fiscal_year'] = fyr
dfu3['fiscal_mo'] = fmo
dfu3.head()

,site_id,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage,fiscal_year,fiscal_mo
0,03,Electricity,2010,7,Electricity charge,-,24.137931,35.0,0.000000,2011,1
1,03,Electricity,2010,7,Electricity charge,kWh,1304.274655,9.5,6656.551724,2011,1
2,03,Electricity,2010,8,Electricity charge,-,39.498433,62.0,0.000000,2011,2
3,03,Electricity,2010,8,Electricity charge,kWh,4940.646254,31.0,26822.236155,2011,2
4,03,Electricity,2010,9,Electricity charge,-,16.363636,27.0,0.000000,2011,3


In [40]:
mmbtu = []
for ix, row in dfu3.iterrows():
    mmbtu.append(
        ut.fuel_btus_per_unit(row.service_type, row.units) * row.usage / 1e6
    )
dfu3['mmbtu'] = mmbtu
dfu3.head(10)

,site_id,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage,fiscal_year,fiscal_mo,mmbtu
0,03,Electricity,2010,7,Electricity charge,-,24.137931,35.0,0.000000,2011,1,NaN
1,03,Electricity,2010,7,Electricity charge,kWh,1304.274655,9.5,6656.551724,2011,1,22.712154
2,03,Electricity,2010,8,Electricity charge,-,39.498433,62.0,0.000000,2011,2,NaN
3,03,Electricity,2010,8,Electricity charge,kWh,4940.646254,31.0,26822.236155,2011,2,91.517470
4,03,Electricity,2010,9,Electricity charge,-,16.363636,27.0,0.000000,2011,3,NaN
5,03,Electricity,2010,9,Electricity charge,kWh,6206.509032,63.0,36144.268383,2011,3,123.324244
6,03,Electricity,2010,10,Electricity charge,kWh,7637.579025,102.0,45079.667877,2011,4,153.811827
7,03,Electricity,2010,11,Electricity charge,kWh,8051.234399,90.0,47705.533154,2011,5,162.771279
8,03,Electricity,2010,12,Electricity charge,kWh,7733.783390,93.0,44821.714849,2011,6,152.931691
9,03,Electricity,2011,1,Electricity charge,kWh,6295.576913,93.0,35977.482508,2011,7,122.755170


In [42]:
dfu3[dfu3.service_type=='Electricity'].head(10)

,site_id,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage,fiscal_year,fiscal_mo,mmbtu
0,03,Electricity,2010,7,Electricity charge,-,24.137931,35.0,0.000000,2011,1,NaN
1,03,Electricity,2010,7,Electricity charge,kWh,1304.274655,9.5,6656.551724,2011,1,22.712154
2,03,Electricity,2010,8,Electricity charge,-,39.498433,62.0,0.000000,2011,2,NaN
3,03,Electricity,2010,8,Electricity charge,kWh,4940.646254,31.0,26822.236155,2011,2,91.517470
4,03,Electricity,2010,9,Electricity charge,-,16.363636,27.0,0.000000,2011,3,NaN
5,03,Electricity,2010,9,Electricity charge,kWh,6206.509032,63.0,36144.268383,2011,3,123.324244
6,03,Electricity,2010,10,Electricity charge,kWh,7637.579025,102.0,45079.667877,2011,4,153.811827
7,03,Electricity,2010,11,Electricity charge,kWh,8051.234399,90.0,47705.533154,2011,5,162.771279
8,03,Electricity,2010,12,Electricity charge,kWh,7733.783390,93.0,44821.714849,2011,6,152.931691
9,03,Electricity,2011,1,Electricity charge,kWh,6295.576913,93.0,35977.482508,2011,7,122.755170


In [43]:
dfu3.to_csv('dfu3.csv')
dfu3.to_pickle('dfu3.pkl')

## Work on Other Utility Functions

### Find All Fiscal Years and Months Present in a DataFrame

In [44]:
bench_util.months_present(dfu3)

[(2009, 5),
 (2009, 6),
 (2009, 7),
 (2009, 8),
 (2009, 9),
 (2009, 10),
 (2009, 11),
 (2009, 12),
 (2010, 1),
 (2010, 2),
 (2010, 3),
 (2010, 4),
 (2010, 5),
 (2010, 6),
 (2010, 7),
 (2010, 8),
 (2010, 9),
 (2010, 10),
 (2010, 11),
 (2010, 12),
 (2011, 1),
 (2011, 2),
 (2011, 3),
 (2011, 4),
 (2011, 5),
 (2011, 6),
 (2011, 7),
 (2011, 8),
 (2011, 9),
 (2011, 10),
 (2011, 11),
 (2011, 12),
 (2012, 1),
 (2012, 2),
 (2012, 3),
 (2012, 4),
 (2012, 5),
 (2012, 6),
 (2012, 7),
 (2012, 8),
 (2012, 9),
 (2012, 10),
 (2012, 11),
 (2012, 12),
 (2013, 1),
 (2013, 2),
 (2013, 3),
 (2013, 4),
 (2013, 5),
 (2013, 6),
 (2013, 7),
 (2013, 8),
 (2013, 9),
 (2013, 10),
 (2013, 11),
 (2013, 12),
 (2014, 1),
 (2014, 2),
 (2014, 3),
 (2014, 4),
 (2014, 5),
 (2014, 6),
 (2014, 7),
 (2014, 8),
 (2014, 9),
 (2014, 10),
 (2014, 11),
 (2014, 12),
 (2015, 1),
 (2015, 2),
 (2015, 3),
 (2015, 4),
 (2015, 5),
 (2015, 6),
 (2015, 7),
 (2015, 8),
 (2015, 9),
 (2015, 10),
 (2015, 11),
 (2015, 12),
 (2016, 1),
 (2016,

In [45]:
bench_util.months_present(dfu3, 'cal_year', 'cal_mo')[-5:]

[(2017, 8), (2017, 9), (2017, 10), (2017, 11), (2017, 12)]